In [ ]:
import glob
import sys
import warnings
warnings.filterwarnings('ignore')

import xarray as xr
import cmocean

sys.path.append('/g/data/xv83/quantile-mapping/qq-workflows')
import validation
sys.path.append('/g/data/xv83/quantile-mapping/qqscale')
import utils

In [ ]:
# Required parameters

assert 'model' in locals(), "Must provide the name of a GCM (option -p model {model-name})"

In [ ]:
def get_obs_files(obs, var):
    """Get obs data"""

    if obs == 'BARRA-R2':
        if var in ['hursmin', 'hursmax']:
            target_files = sorted(glob.glob(f'/g/data/ia39/australian-climate-service/test-data/observations/BARRA2/output/reanalysis/AUS-11/BOM/ERA5/historical/hres/BARRA-R2/v1/day/{var}/v20240516/*.nc'))
        else:
            target_files = sorted(glob.glob(f"/g/data/ob53/BARRA2/output/reanalysis/AUS-11/BOM/ERA5/historical/hres/BARRA-R2/v1/day/{var}/latest/*.nc"))

    return target_files[72:432]
    
    
def get_model_files(model, var):
    """Get model data"""
    
    if model == "ACCESS-CM2":
        hist_files = sorted(glob.glob(f"/g/data/fs38/publications/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r4i1p1f1/day/{var}/gn/latest/*.nc"))
    elif model == "EC-Earth3":
        hist_files = sorted(glob.glob(f"/g/data/oi10/replicas/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3/historical/r1i1p1f1/day/{var}/gr/v20200310/*.nc"))        
    elif model == "CMCC-ESM2":
        hist_files = sorted(glob.glob(f"/g/data/oi10/replicas/CMIP6/CMIP/CMCC/CMCC-ESM2/historical/r1i1p1f1/day/{var}/gn/v20210114/*.nc"))        
          
    return hist_files

In [ ]:
hist_units = {
    'rsds': "W m-2",
    'tasmin': 'K',
    'tasmax': 'K',
    'hursmin': '%',
    'hursmax': '%',
    'sfcWind': 'm s-1',
    'sfcWindmax': 'm s-1',
}

target_units = {
    'rsds': "W m-2",
    'tasmin': 'K',
    'tasmax': 'K',
    'hursmin': '%',
    'hursmax': '%',
    'sfcWind': 'm s-1',
    'sfcWindmax': 'm s-1',
}

output_units = {
    'rsds': "W m-2",
    'tasmin': 'C',
    'tasmax': 'C',
    'hursmin': '%',
    'hursmax': '%',
    'sfcWind': 'm s-1',
    'sfcWindmax': 'm s-1',
}

clim_cbar = {
    'rsds': cmocean.cm.solar,
    'tasmin': cmocean.cm.thermal,
    'tasmax': cmocean.cm.thermal,
    'hursmin': cmocean.cm.thermal,
    'hursmax': cmocean.cm.thermal,
    'sfcWind': cmocean.cm.speed,
    'sfcWindmax': cmocean.cm.speed,
}

clim_levels = {
    'rsds': [60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280, 300, 320, 340],
    'tasmin': [-2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30],
    'tasmax': [5, 8, 11, 14, 17, 20, 23, 26, 29, 31, 34, 37, 40],
    'hursmin': [5, 10, 15 ,20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80],
    'hursmax': [25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95],
    'sfcWind': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'sfcWindmax': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
}

diff_cbar = {
    'rsds': 'RdBu_r',
    'tasmin': 'RdBu_r',
    'tasmax': 'RdBu_r',
    'hursmin': 'RdBu_r',
    'hursmax': 'RdBu_r',
    'sfcWind': 'RdBu_r',
    'sfcWindmax': 'RdBu_r',
}

ratio_levels = [-40, -35, -30, -25, -20, -15, -10, -5, 0, 5, 10, 15, 20, 25, 30, 35, 40]

diff_levels = {
    'rsds': [-18, -15, -12, -9, -6, -3, 0, 3, 6, 9, 12, 15, 18],
    'tasmin': [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
    'tasmax': [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
    'hursmin': [-12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12],
    'hursmax': [-12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12],
    'sfcWind': [-2.0, -1.6, -1.2, -0.8, -0.4, 0, 0.4, 0.8, 1.2, 1.6, 2.0],
    'sfcWindmax': [-2.0, -1.6, -1.2, -0.8, -0.4, 0, 0.4, 0.8, 1.2, 1.6, 2.0],
}

In [ ]:
def get_data(var, model, obs, quantiles=[0.25, 0.5, 0.75]):
    """Plot the comparison"""

    hist_files = get_model_files(model, var)
    ds_hist = utils.read_data(
        hist_files,
        var,
        time_bounds=["1985", "2014"],
        input_units=hist_units[var],
        output_units=output_units[var],
    )
    hist_pcts = ds_hist[var].quantile(quantiles, dim='time', keep_attrs=True).compute()
    
    target_files = get_obs_files(obs, var)
    ds_target = utils.read_data(
        target_files,
        var,
        time_bounds=["1985", "2014"],
        input_units=target_units[var],
        output_units=output_units[var],
    )
    try:
        ds_target = ds_target.drop_vars('crs')
    except ValueError:
        pass
    target_pcts = ds_target[var].quantile(quantiles, dim='time', keep_attrs=True).compute()
    
    comparison_ratio = validation.spatial_comparison_data(
        target_pcts,
        hist_pcts,
        'multiplicative'
    )
    comparison_diff = validation.spatial_comparison_data(
        target_pcts,
        hist_pcts,
        'additive'
    )
    
    return hist_pcts, target_pcts, comparison_diff, comparison_ratio

### rsds

In [ ]:
hist_pcts, target_pcts, comparison_diff, comparison_ratio = get_data("rsds", model, "BARRA-R2")

In [ ]:
for quantile in [0.25, 0.5, 0.75]:
    percentile = quantile * 100
    print(f'percentile: {percentile}')
    validation.spatial_comparison_plot(
        target_pcts.sel(quantile=quantile),
        hist_pcts.sel(quantile=quantile),
        comparison_diff.sel(quantile=quantile),
        'BARRA-R2',
        'ACCESS-CM2',
        clim_cbar["rsds"],
        diff_cbar["rsds"],
        clim_levels["rsds"],
        diff_levels["rsds"],
        'additive',
        land_only=True,
        clim_extend='neither',
        print_mav=True,
    )

In [ ]:
for quantile in [0.25, 0.5, 0.75]:
    percentile = quantile * 100
    print(f'percentile: {percentile}')
    validation.spatial_comparison_plot(
        target_pcts.sel(quantile=quantile),
        hist_pcts.sel(quantile=quantile),
        comparison_ratio.sel(quantile=quantile),
        'BARRA-R2',
        'ACCESS-CM2',
        clim_cbar["rsds"],
        diff_cbar["rsds"],
        clim_levels["rsds"],
        ratio_levels,
        'multiplicative',
        land_only=True,
        clim_extend='neither',
        print_mav=True,
    )

### tasmin

In [ ]:
hist_pcts, target_pcts, comparison_diff, comparison_ratio = get_data("tasmin", model, "BARRA-R2")

In [ ]:
for quantile in [0.25, 0.5, 0.75]:
    percentile = quantile * 100
    print(f'percentile: {percentile}')
    validation.spatial_comparison_plot(
        target_pcts.sel(quantile=quantile),
        hist_pcts.sel(quantile=quantile),
        comparison_diff.sel(quantile=quantile),
        'BARRA-R2',
        'ACCESS-CM2',
        clim_cbar["tasmin"],
        diff_cbar["tasmin"],
        clim_levels["tasmin"],
        diff_levels["tasmin"],
        'additive',
        land_only=True,
        clim_extend='neither',
        print_mav=True,
    )

In [ ]:
for quantile in [0.25, 0.5, 0.75]:
    percentile = quantile * 100
    print(f'percentile: {percentile}')
    validation.spatial_comparison_plot(
        target_pcts.sel(quantile=quantile),
        hist_pcts.sel(quantile=quantile),
        comparison_ratio.sel(quantile=quantile),
        'BARRA-R2',
        'ACCESS-CM2',
        clim_cbar["tasmin"],
        diff_cbar["tasmin"],
        clim_levels["tasmin"],
        ratio_levels,
        'multiplicative',
        land_only=True,
        clim_extend='neither',
        print_mav=True,
    )

### tasmax

In [ ]:
hist_pcts, target_pcts, comparison_diff, comparison_ratio = get_data("tasmax", model, "BARRA-R2")

In [ ]:
for quantile in [0.25, 0.5, 0.75]:
    percentile = quantile * 100
    print(f'percentile: {percentile}')
    validation.spatial_comparison_plot(
        target_pcts.sel(quantile=quantile),
        hist_pcts.sel(quantile=quantile),
        comparison_diff.sel(quantile=quantile),
        'BARRA-R2',
        'ACCESS-CM2',
        clim_cbar["tasmax"],
        diff_cbar["tasmax"],
        clim_levels["tasmax"],
        diff_levels["tasmax"],
        'additive',
        land_only=True,
        clim_extend='neither',
        print_mav=True,
    )

In [ ]:
for quantile in [0.25, 0.5, 0.75]:
    percentile = quantile * 100
    print(f'percentile: {percentile}')
    validation.spatial_comparison_plot(
        target_pcts.sel(quantile=quantile),
        hist_pcts.sel(quantile=quantile),
        comparison_ratio.sel(quantile=quantile),
        'BARRA-R2',
        'ACCESS-CM2',
        clim_cbar["tasmax"],
        diff_cbar["tasmax"],
        clim_levels["tasmax"],
        ratio_levels,
        'multiplicative',
        land_only=True,
        clim_extend='neither',
        print_mav=True,
    )

### sfcWind

In [ ]:
hist_pcts, target_pcts, comparison_diff, comparison_ratio = get_data("sfcWind", model, "BARRA-R2")

In [ ]:
for quantile in [0.25, 0.5, 0.75]:
    percentile = quantile * 100
    print(f'percentile: {percentile}')
    validation.spatial_comparison_plot(
        target_pcts.sel(quantile=quantile),
        hist_pcts.sel(quantile=quantile),
        comparison_diff.sel(quantile=quantile),
        'BARRA-R2',
        'ACCESS-CM2',
        clim_cbar["sfcWind"],
        diff_cbar["sfcWind"],
        clim_levels["sfcWind"],
        diff_levels["sfcWind"],
        'additive',
        land_only=True,
        clim_extend='neither',
        print_mav=True,
    )

In [ ]:
for quantile in [0.25, 0.5, 0.75]:
    percentile = quantile * 100
    print(f'percentile: {percentile}')
    validation.spatial_comparison_plot(
        target_pcts.sel(quantile=quantile),
        hist_pcts.sel(quantile=quantile),
        comparison_ratio.sel(quantile=quantile),
        'BARRA-R2',
        'ACCESS-CM2',
        clim_cbar["sfcWind"],
        diff_cbar["sfcWind"],
        clim_levels["sfcWind"],
        ratio_levels,
        'multiplicative',
        land_only=True,
        clim_extend='neither',
        print_mav=True,
    )

### sfcWindmax

In [ ]:
hist_pcts, target_pcts, comparison_diff, comparison_ratio = get_data("sfcWindmax", model, "BARRA-R2")

In [ ]:
for quantile in [0.25, 0.5, 0.75]:
    percentile = quantile * 100
    print(f'percentile: {percentile}')
    validation.spatial_comparison_plot(
        target_pcts.sel(quantile=quantile),
        hist_pcts.sel(quantile=quantile),
        comparison_diff.sel(quantile=quantile),
        'BARRA-R2',
        'ACCESS-CM2',
        clim_cbar["sfcWindmax"],
        diff_cbar["sfcWindmax"],
        clim_levels["sfcWindmax"],
        diff_levels["sfcWindmax"],
        'additive',
        land_only=True,
        clim_extend='neither',
        print_mav=True,
    )

In [ ]:
for quantile in [0.25, 0.5, 0.75]:
    percentile = quantile * 100
    print(f'percentile: {percentile}')
    validation.spatial_comparison_plot(
        target_pcts.sel(quantile=quantile),
        hist_pcts.sel(quantile=quantile),
        comparison_ratio.sel(quantile=quantile),
        'BARRA-R2',
        'ACCESS-CM2',
        clim_cbar["sfcWindmax"],
        diff_cbar["sfcWindmax"],
        clim_levels["sfcWindmax"],
        ratio_levels,
        'multiplicative',
        land_only=True,
        clim_extend='neither',
        print_mav=True,
    )

### hursmin

In [ ]:
hist_pcts, target_pcts, comparison_diff, comparison_ratio = get_data("hursmin", model, "BARRA-R2")

In [ ]:
for quantile in [0.25, 0.5, 0.75]:
    percentile = quantile * 100
    print(f'percentile: {percentile}')
    validation.spatial_comparison_plot(
        target_pcts.sel(quantile=quantile),
        hist_pcts.sel(quantile=quantile),
        comparison_diff.sel(quantile=quantile),
        'BARRA-R2',
        'ACCESS-CM2',
        clim_cbar["hursmin"],
        diff_cbar["hursmin"],
        clim_levels["hursmin"],
        diff_levels["hursmin"],
        'additive',
        land_only=True,
        clim_extend='neither',
        print_mav=True,
    )

In [ ]:
for quantile in [0.25, 0.5, 0.75]:
    percentile = quantile * 100
    print(f'percentile: {percentile}')
    validation.spatial_comparison_plot(
        target_pcts.sel(quantile=quantile),
        hist_pcts.sel(quantile=quantile),
        comparison_ratio.sel(quantile=quantile),
        'BARRA-R2',
        'ACCESS-CM2',
        clim_cbar["hursmin"],
        diff_cbar["hursmin"],
        clim_levels["hursmin"],
        ratio_levels,
        'multiplicative',
        land_only=True,
        clim_extend='neither',
        print_mav=True,
    )

### hursmax

In [ ]:
hist_pcts, target_pcts, comparison_diff, comparison_ratio = get_data("hursmax", model, "BARRA-R2")

In [ ]:
for quantile in [0.25, 0.5, 0.75]:
    percentile = quantile * 100
    print(f'percentile: {percentile}')
    validation.spatial_comparison_plot(
        target_pcts.sel(quantile=quantile),
        hist_pcts.sel(quantile=quantile),
        comparison_diff.sel(quantile=quantile),
        'BARRA-R2',
        'ACCESS-CM2',
        clim_cbar["hursmax"],
        diff_cbar["hursmax"],
        clim_levels["hursmax"],
        diff_levels["hursmax"],
        'additive',
        land_only=True,
        clim_extend='max',
        print_mav=True,
    )

In [ ]:
for quantile in [0.25, 0.5, 0.75]:
    percentile = quantile * 100
    print(f'percentile: {percentile}')
    validation.spatial_comparison_plot(
        target_pcts.sel(quantile=quantile),
        hist_pcts.sel(quantile=quantile),
        comparison_ratio.sel(quantile=quantile),
        'BARRA-R2',
        'ACCESS-CM2',
        clim_cbar["hursmax"],
        diff_cbar["hursmax"],
        clim_levels["hursmax"],
        ratio_levels,
        'multiplicative',
        land_only=True,
        clim_extend='max',
        print_mav=True,
    )